Using FinBERT to run sentiment on all texts of articles

In [ ]:
!pip install -q transformers
!pip install -q newspaper3k pandas numpy
!pip install -q lxml-html-clean
!pip install -q spacy

  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.4/7.4 MB 41.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.1/211.1 kB 31.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.3/81.3 kB 13.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 104.9/104.9 kB 16.0 MB/s eta 0:00:00


In [ ]:
!python -m spacy download en_core_web_trf

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.2/1.2 MB 83.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.5/72.5 kB 13.1 MB/s eta 0:00:00
  Attempting uninstall: wheel
    Found existing installation: wheel 0.44.0
    Uninstalling wheel-0.44.0:
      Successfully uninstalled wheel-0.44.0
  Attempting uninstall: setuptools
    Found existing installation: setuptools 69.5.1
    Uninstalling setuptools-69.5.1:
      Successfully uninstalled setuptools-69.5.1
  Attempting uninstall: pip
    Found existing installation: pip 24.1.2
    Uninstalling pip-24.1.2:
      Successfully uninstalled pip-24.1.2
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
ipython 7.34.0 requires jedi>=0.16, which is not installed.


ERROR: Invalid requirement: '“spacy[cuda116]”': Expected package name at the start of dependency specifier
    “spacy[cuda116]”
    ^
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 457.4/457.4 MB 66.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 731.6/731.6 kB 16.3 MB/s eta 0:00:00
✔ Download and installation successful
You can now load the package via spacy.load('en_core_web_trf')
⚠ Restart to reload dependencies
If you are in a Jupyter or Colab notebook, you may need to restart Python in
order to load all the package's dependencies. You can do this by selecting the
'Restart kernel' or 'Restart runtime' option.


In [ ]:
import pandas as pd
import numpy as np
import torch
from newspaper import Article
import os
import spacy


In [ ]:
spacy.require_gpu()

True

In [ ]:
nlp = spacy.load("en_core_web_trf")

/usr/local/lib/python3.10/dist-packages/spacy/util.py:1740: UserWarning: [W111] Jupyter notebook detected: if using `prefer_gpu()` or `require_gpu()`, include it in the same cell right before `spacy.load()` to ensure that the model is loaded on the correct device. More information: http://spacy.io/usage/v3#jupyter-notebook-gpu
  warnings.warn(Warnings.W111)
/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:253: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode un

In [ ]:
txt = "Adherents of the Dogs strategy assume that the highest yielding Dow stocks are unfairly and temporarily depressed. That presumably means they'll hold up better than the S&P 500. And that seems to be the case in 2022. Adding the Dogs stocks' 4% average dividend yield coming into 2022 plus their average 1.8% drop still left investors with a positive return of more than 2% on the stocks. Additionally, half of last year's Dogs, including International Business Machines (IBM), Chevron (CVX), Merck (MRK), Amgen (AMGN), and Coca-Cola (KO) posted gains in 2022. Chevron alone gained 53% on top of yielding 4.6%."
doc = nlp(txt)

for ent in doc.ents:
    print(ent.text, ent.start_char, ent.end_char, ent.label_)

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Dow 64 67 ORG
2022 211 215 DATE
4% 241 243 PERCENT
2022 279 283 DATE
1.8% 303 307 PERCENT
more than 2% 360 372 PERCENT
half 402 406 CARDINAL
last year's 410 421 DATE
International Business Machines 438 469 ORG
IBM 471 474 ORG
Chevron 477 484 ORG
CVX 486 489 ORG
Merck 492 497 ORG
MRK 499 502 ORG
Amgen 505 510 ORG
AMGN 512 516 ORG
Coca-Cola 523 532 ORG
KO 534 536 ORG
2022 554 558 DATE
Chevron 560 567 ORG
53% 581 584 PERCENT
4.6% 604 608 PERCENT


In [ ]:
print(torch.cuda.is_available())
print(torch.cuda.device_count())

True
1


In [ ]:
!ls /content/old_data

AAPL_main.csv	   GOOGL_main.csv      INTC_main.csv	  MSFT_main.csv      NVDA_main.csv
AAPL_relation.csv  GOOGL_relation.csv  INTC_relation.csv  MSFT_relation.csv  NVDA_relation.csv
AMZN_main.csv	   IBM_main.csv        META_main.csv	  NFLX_main.csv      ORCL_main.csv
AMZN_relation.csv  IBM_relation.csv    META_relation.csv  NFLX_relation.csv  ORCL_relation.csv


In [ ]:
from transformers import pipeline

pipe = pipeline("sentiment-analysis", model="ProsusAI/finbert", device=0 if torch.cuda.is_available() else -1)

config.json:   0%|          | 0.00/758 [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/438M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/252 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

In [ ]:
from google.colab import files
uploaded = files.upload()

Saving AAPL_main.csv to AAPL_main.csv
Saving AAPL_relation.csv to AAPL_relation.csv


In [ ]:
def get_news_text(url):
    article = Article(url)
    try:
        article.download()
        article.parse()
        return article.text
    except Exception as error:
        return error

In [ ]:
def split_text_by_ticker_aliases(text, nlp, ticker_aliases):
    """
    Splits the text into sections based on the presence of ticker names or their aliases,
    as well as a category for text not related to any ticker.
    """
    split_text = {ticker: list([]) for ticker in ticker_aliases.keys()}
    split_text["none"] = list([])  # For text unrelated to any ticker

    # alias_to_ticker = {}
    # for ticker, aliases in ticker_aliases.items():
    #     for alias in aliases:
    #         alias_to_ticker[alias.lower()] = ticker  # Use lowercase for matching

    doc = nlp(text)

    sentences = list(doc.sents)

    for sentence in sentences:
        associated = False

        # Check for ticker matches in named entities
        for ent in sentence.ents:
            alias = ent.text.lower()
            for ticker, aliases in ticker_aliases.items():
                for aliase in aliases:
                    if alias in aliase.lower() or aliase.lower() in alias:
                        split_text[ticker].append(sentence.text)
                        associated = True
                        break


            # if alias in alias_to_ticker:
            #     ticker = alias_to_ticker[alias]
            #     split_text[ticker] += sentence.text + " "
            #     associated = True

        if not associated:
            split_text["none"].append(sentence.text)

    return split_text

In [ ]:
def combine_dataframes_with_time_range(main_df, relation_df, start_time, end_time):
    start_time = pd.to_datetime(start_time)
    end_time = pd.to_datetime(end_time)

    main_df['publish_time'] = pd.to_datetime(main_df['publish_time'])
    relation_df['time'] = pd.to_datetime(relation_df['time'])
    filtered_rel = relation_df[relation_df['ticker'] == relation_df['source_ticker']]

    # Merge the two dataframes on `id` and `news_id`
    merged_df = main_df.merge(filtered_rel, left_on='id', right_on='news_id', how='inner')

    filtered_df = merged_df

    time_filtered_df = filtered_df[(filtered_df['publish_time'] >= start_time) & (filtered_df['publish_time'] <= end_time)]

    time_filtered_df = time_filtered_df.rename(columns={'ticker_x': 'ticker'})

    result_df = time_filtered_df[['id', 'publish_time', 'article_url', 'sentiment', 'ticker']]

    return result_df

In [ ]:
def analyze_sentiments_with_aliases(
    merged_df, pipe, nlp, ticker_aliases, truncate_or_split="truncate", skip_none=True
):
    """
    Perform sentiment analysis on financial news articles grouped by ticker aliases.
    """
    result_data = []

    for i, row in enumerate(merged_df.itertuples(), start=1):
        if i % 500 == 0:
            print(f"Processing article {i}/{len(merged_df)}")

        url = row.article_url
        text = get_news_text(url)

        if text != "Error" and isinstance(text, str):
            split_texts = split_text_by_ticker_aliases(text, nlp, ticker_aliases)

            for ticker, sentences in split_texts.items():
                if not sentences:  # Skip if no sentences for the ticker
                    continue
                if skip_none and ticker == "none":
                    continue

                combined_text = " ".join(sentences)

                # Process long text based on the chosen strategy
                if truncate_or_split == "split":
                    tokens = pipe.tokenizer(combined_text, truncation=False, add_special_tokens=True)["input_ids"]
                    if len(tokens) > 512:
                        chunk_size = 512
                        chunks = [combined_text[start : start + chunk_size] for start in range(0, len(combined_text), chunk_size)]

                        # Analyze each chunk and compute average score
                        total_score = 0
                        for chunk in chunks:
                            result = pipe(chunk)
                            label = result[0]["label"]
                            score = result[0]["score"]
                            if label == "positive":
                                total_score += score
                            elif label == "negative":
                                total_score -= score
                        # Determine final sentiment
                        predicted_score = total_score / len(chunks)
                        if predicted_score > 0.1:
                            predicted_sentiment = "positive"
                        elif predicted_score < -0.1:
                            predicted_sentiment = "negative"
                        else:
                            predicted_sentiment = "neutral"
                    else:
                        # Text is already within token limits
                        result = pipe(combined_text)
                        predicted_sentiment = result[0]["label"]
                        # predicted_score = result[0]["score"]

                        if predicted_sentiment == "positive":
                            predicted_score = result[0]['score']
                        elif predicted_sentiment == "negative":
                            predicted_score = -result[0]['score']
                        else:
                            predicted_score = 0
                else:
                    result = pipe(combined_text, truncation=True)
                    predicted_sentiment = result[0]["label"]
                    # predicted_score = result[0]["score"]

                    if predicted_sentiment == "positive":
                        predicted_score = result[0]['score']
                    elif predicted_sentiment == "negative":
                        predicted_score = -result[0]['score']
                    else:
                        predicted_score = 0

                result_data.append({
                    "ticker": ticker,
                    "source_ticker": row.ticker,
                    "sentiment": predicted_sentiment,
                    "sentiment_score": predicted_score,
                    "news_id": row.id,
                    "time": row.publish_time,
                    "article_url": row.article_url
                })

    result_df = pd.DataFrame(
        result_data,
        columns=["ticker", "source_ticker", "sentiment", "sentiment_score", "news_id", "time", "article_url"]
    )

    return result_df

In [ ]:
target_dir = "/content/old_data"
new_directory = "/content/total_sentiments"
if not os.path.exists(new_directory):
        os.makedirs(new_directory)

In [ ]:
aliases = {
    "META": {
        "META": {"Meta", "Facebook", "Meta Platforms"}
    },
    "AAPL": {
        "AAPL": {"Apple", "AAPL"}
    },
    "AMZN": {
        "AMZN": {"Amazon", "Amazon.com", "AMZN"}
    },
    "NFLX": {
        "NFLX": {"Netflix", "NFLX"}
    },
    "GOOGL": {
        "GOOGL": {"Google", "Alphabet", "GOOGL"}
    },
    "MSFT": {
        "MSFT": {"Microsoft", "MSFT", "Windows", "Azure"}
    },
    "IBM": {
        "IBM": {"IBM", "International Business Machines", "Big Blue"}
    },
    "ORCL": {
        "ORCL": {"Oracle", "ORCL", "Oracle Corporation"}
    },
    "NVDA": {
        "NVDA": {"NVIDIA", "Nvidia", "NVDA", "GeForce"}
    },
    "INTC": {
        "INTC": {"Intel", "INTC", "Intel Corporation"}
    }
}

In [ ]:
start_time = "2024-07-05"
end_time = "2024-07-06"

test_df = pd.read_csv("/content/old_data/AAPL_main.csv")
test_df = test_df[(test_df['publish_time'] >= start_time) & (test_df['publish_time'] <= end_time)]
print(test_df)

                                                    id         publish_time  \
610  aff9799c0acb24e8ad98d379473fb51b323101ef58062a...  2024-07-05 22:30:00   
611  bb887e8804c0133d0a9b4dce216c6384551a34981e8cfb...  2024-07-05 21:35:51   
612  9f32fe77fa214130cb349e4134148ebceb8902bddfe65b...  2024-07-05 20:30:00   
613  c48c3a6397fcddafd33de8a5a3b0928ff0ac1e163fda5b...  2024-07-05 20:06:45   
614  12db91dff680f61d476a9f48cc4167f37f4331364f778b...  2024-07-05 19:20:06   
615  fec6c1e85cb3877391e7f0509ae67e00d9e302b3dda2a5...  2024-07-05 14:23:52   
616  6d66f0e3700b559ef5f87c171fe04788896afff13b4451...  2024-07-05 14:04:41   
617  d8cfa6720e0d1e79a26b8b2106a584e0ed0504410caa51...  2024-07-05 14:00:53   
618  390ca9f6773dbc087a1627354840a24265f498d58ee328...  2024-07-05 13:57:56   
619  a85adbfb3047c68c9d64a36e7fb41c2c73bf82f5003cd3...  2024-07-05 13:25:32   
620  4dbae762ac5a316f674c52e59dfc823ac508f360452d1f...  2024-07-05 10:21:11   
621  9e54e80bd1c0b817e3ac110f34f7704262194794080b98.

In [ ]:
result_df = analyze_sentiments_with_aliases(test_df, pipe, nlp, aliases["AAPL"], truncate_or_split="split")

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Processing article 10/18


In [ ]:
print(result_df)

   ticker source_ticker sentiment  sentiment_score  \
0    AAPL          AAPL  positive         0.861411   
1    AAPL          AAPL   neutral         0.000000   
2    AAPL          AAPL   neutral         0.000000   
3    AAPL          AAPL  positive         0.840944   
4    AAPL          AAPL  negative        -0.968945   
5    AAPL          AAPL  positive         0.647968   
6    AAPL          AAPL  negative        -0.429588   
7    AAPL          AAPL  positive         0.647968   
8    AAPL          AAPL   neutral         0.000000   
9    AAPL          AAPL   neutral         0.000000   
10   AAPL          AAPL  positive         0.835910   
11   AAPL          AAPL  positive         0.835910   
12   AAPL          AAPL  positive         0.637347   
13   AAPL          AAPL   neutral         0.000000   

                                              news_id                 time  \
0   aff9799c0acb24e8ad98d379473fb51b323101ef58062a...  2024-07-05 22:30:00   
1   bb887e8804c0133d0a9b4dce216c6

In [ ]:
# one file takes too long, split them up
for file_name in os.listdir(target_dir):
    file_path = os.path.join(target_dir, file_name)
    if os.path.isfile(file_path):  # Check if it is a file
        if "main" in file_name:
            # print(file_name)
            ticker = file_name.split("_")[0]
            print(ticker)
            df_news = pd.read_csv(file_path)

            result_df = analyze_sentiments_with_aliases(df_news, pipe, nlp, aliases[ticker], truncate_or_split="split")
            result_df.to_csv(f"{new_directory}/{ticker}_total_sentiments.csv", index=False)

In [ ]:
# IBM:
tick = "IBM"
input_df = pd.read_csv(f"{target_dir}/{tick}_main.csv")
result_df = analyze_sentiments_with_aliases(input_df, pipe, nlp, aliases[tick], truncate_or_split="split")

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Processing article 500/1991
Processing article 1000/1991
Processing article 1500/1991


In [ ]:
print(result_df)

    ticker source_ticker sentiment  sentiment_score  \
0      IBM           IBM  negative        -0.920723   
1      IBM           IBM   neutral         0.000000   
2      IBM           IBM  positive         0.275779   
3      IBM           IBM  positive         0.702788   
4      IBM           IBM  negative        -0.973019   
..     ...           ...       ...              ...   
794    IBM           IBM   neutral         0.000000   
795    IBM           IBM  positive         0.788304   
796    IBM           IBM   neutral         0.000000   
797    IBM           IBM  positive         0.940293   
798    IBM           IBM  positive         0.939638   

                                               news_id                 time  \
0    ac68640c4bd892c57b9769569b9021060c7803657044de...  2024-10-28 22:30:00   
1    e79a5e50fb91da1c095621f9151c02319747a2a0785edc...  2024-10-25 14:44:00   
2    270d2e6f5e0851947f4203ea9b12003cddef813da4d954...  2024-10-25 14:32:00   
3    1a28db18f6d90d339c

In [ ]:
result_df.to_csv(f"{new_directory}/{tick}_total_sentiments.csv", index=False)

In [ ]:
!ls /content/total_sentiments

IBM_total_sentiments.csv   META_total_sentiments.csv  ORCL_total_sentiments.csv
INTC_total_sentiments.csv  NFLX_total_sentiments.csv


In [ ]:
files.download(f"{new_directory}/IBM_total_sentiments.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# INTC:
tick = "INTC"
input_df = pd.read_csv(f"{target_dir}/{tick}_main.csv")
result_df = analyze_sentiments_with_aliases(input_df, pipe, nlp, aliases[tick], truncate_or_split="split")

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Processing article 500/2989
Processing article 1000/2989
Processing article 1500/2989
Processing article 2000/2989
Processing article 2500/2989


In [ ]:
print(result_df)

     ticker source_ticker sentiment  sentiment_score  \
0      INTC          INTC  positive         0.944777   
1      INTC          INTC  positive         0.663926   
2      INTC          INTC  negative        -0.968407   
3      INTC          INTC   neutral         0.000000   
4      INTC          INTC  positive         0.922391   
...     ...           ...       ...              ...   
1515   INTC          INTC  positive         0.525131   
1516   INTC          INTC  negative        -0.599807   
1517   INTC          INTC  positive         0.489739   
1518   INTC          INTC   neutral         0.000000   
1519   INTC          INTC  negative        -0.480419   

                                                news_id                 time  \
0     7377ea685f9a89bd6ccaae3aa4635e5ffe01245e8634fe...  2024-10-30 09:45:00   
1     9fd75f3f763b0ce73c4a20aa8eca75a71c02242b524864...  2024-10-29 11:45:00   
2     4fe36e1b97aaab0acff95403c29fffb7c75436de4197ec...  2024-10-29 08:25:00   
3     7

In [ ]:
result_df.to_csv(f"{new_directory}/{tick}_total_sentiments.csv", index=False)

In [ ]:
files.download(f"{new_directory}/INTC_total_sentiments.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# META:
tick = "META"
input_df = pd.read_csv(f"{target_dir}/{tick}_main.csv")
result_df = analyze_sentiments_with_aliases(input_df, pipe, nlp, aliases[tick], truncate_or_split="split")


/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Processing article 500/6957
Processing article 1000/6957
Processing article 1500/6957
Processing article 2000/6957
Processing article 2500/6957
Processing article 3000/6957
Processing article 3500/6957
Processing article 4000/6957
Processing article 4500/6957
Processing article 5000/6957
Processing article 5500/6957
Processing article 6000/6957
Processing article 6500/6957


In [ ]:
print(result_df)

     ticker source_ticker sentiment  sentiment_score  \
0      META          META   neutral         0.000000   
1      META          META   neutral         0.000000   
2      META          META   neutral         0.000000   
3      META          META   neutral         0.000000   
4      META          META   neutral         0.000000   
...     ...           ...       ...              ...   
2855   META          META   neutral         0.027943   
2856   META          META  positive         0.957545   
2857   META          META   neutral         0.000000   
2858   META          META   neutral         0.000000   
2859   META          META   neutral         0.000000   

                                                news_id                 time  \
0     1d9a233266513e7565c8aa26700bc8a7fad1acb0eb8f8f...  2024-10-31 04:26:28   
1     40f6f17a31ea6dce828171990cd72377669fa26cabd02b...  2024-10-30 08:57:00   
2     73b4269117fe7fd0fa590192ba9f9f02d679451ed01ba3...  2024-10-28 13:12:33   
3     f

In [ ]:
result_df.to_csv(f"{new_directory}/{tick}_total_sentiments.csv", index=False)

In [ ]:
files.download(f"{new_directory}/META_total_sentiments.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
!ls /content/total_sentiments

IBM_total_sentiments.csv  INTC_total_sentiments.csv  META_total_sentiments.csv


In [ ]:
# NFLX:
tick = "NFLX"
input_df = pd.read_csv(f"{target_dir}/{tick}_main.csv")
result_df = analyze_sentiments_with_aliases(input_df, pipe, nlp, aliases[tick], truncate_or_split="split")

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Processing article 500/2994
Processing article 1000/2994
Processing article 1500/2994
Processing article 2000/2994
Processing article 2500/2994


In [ ]:
result_df.to_csv(f"{new_directory}/{tick}_total_sentiments.csv", index=False)

In [ ]:
files.download(f"{new_directory}/{tick}_total_sentiments.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# ORCL:
tick = "ORCL"
input_df = pd.read_csv(f"{target_dir}/{tick}_main.csv")
result_df = analyze_sentiments_with_aliases(input_df, pipe, nlp, aliases[tick], truncate_or_split="split")

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Processing article 500/1996
Processing article 1000/1996
Processing article 1500/1996


In [ ]:
result_df.to_csv(f"{new_directory}/{tick}_total_sentiments.csv", index=False)

In [ ]:
files.download(f"{new_directory}/{tick}_total_sentiments.csv")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
# GOOGL:
tick = "GOOGL"
input_df = pd.read_csv(f"{target_dir}/{tick}_main.csv")
result_df = analyze_sentiments_with_aliases(input_df, pipe, nlp, aliases[tick], truncate_or_split="split")
result_df.to_csv(f"{new_directory}/{tick}_total_sentiments.csv", index=False)
files.download(f"{new_directory}/{tick}_total_sentiments.csv")

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Processing article 500/7926
Processing article 1000/7926
Processing article 1500/7926
Processing article 2000/7926
Processing article 2500/7926
Processing article 3000/7926
Processing article 3500/7926
Processing article 4000/7926
Processing article 4500/7926
Processing article 5000/7926
Processing article 5500/7926
Processing article 6000/7926
Processing article 6500/7926
Processing article 7000/7926
Processing article 7500/7926


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [ ]:
print(len(result_df))

3977


In [64]:
!ls /content/total_sentiments

GOOGL_total_sentiments.csv  INTC_total_sentiments.csv  NFLX_total_sentiments.csv
IBM_total_sentiments.csv    META_total_sentiments.csv  ORCL_total_sentiments.csv


In [65]:
# AAPL:
tick = "AAPL"
input_df = pd.read_csv(f"{target_dir}/{tick}_main.csv")
result_df = analyze_sentiments_with_aliases(input_df, pipe, nlp, aliases[tick], truncate_or_split="split")
result_df.to_csv(f"{new_directory}/{tick}_total_sentiments.csv", index=False)
files.download(f"{new_directory}/{tick}_total_sentiments.csv")

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Processing article 500/9881
Processing article 1000/9881
Processing article 1500/9881
Processing article 2000/9881
Processing article 2500/9881
Processing article 3000/9881
Processing article 3500/9881
Processing article 4000/9881
Processing article 4500/9881
Processing article 5000/9881
Processing article 5500/9881
Processing article 6000/9881
Processing article 6500/9881
Processing article 7000/9881
Processing article 7500/9881
Processing article 8000/9881
Processing article 8500/9881
Processing article 9000/9881
Processing article 9500/9881


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [66]:
# AMZN:
tick = "AMZN"
input_df = pd.read_csv(f"{target_dir}/{tick}_main.csv")
result_df = analyze_sentiments_with_aliases(input_df, pipe, nlp, aliases[tick], truncate_or_split="split")
result_df.to_csv(f"{new_directory}/{tick}_total_sentiments.csv", index=False)
files.download(f"{new_directory}/{tick}_total_sentiments.csv")

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):


Processing article 500/10885
Processing article 1000/10885
Processing article 1500/10885
Processing article 2000/10885
Processing article 2500/10885
Processing article 3000/10885
Processing article 3500/10885
Processing article 4000/10885
Processing article 4500/10885
Processing article 5000/10885
Processing article 5500/10885
Processing article 6000/10885
Processing article 6500/10885
Processing article 7000/10885
Processing article 7500/10885
Processing article 8000/10885
Processing article 8500/10885
Processing article 9000/10885
Processing article 9500/10885
Processing article 10000/10885
Processing article 10500/10885


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [67]:
!ls /content/total_sentiments

AAPL_total_sentiments.csv   IBM_total_sentiments.csv   NFLX_total_sentiments.csv
AMZN_total_sentiments.csv   INTC_total_sentiments.csv  ORCL_total_sentiments.csv
GOOGL_total_sentiments.csv  META_total_sentiments.csv


In [ ]:
# MSFT:
tick = "MSFT"
input_df = pd.read_csv(f"{target_dir}/{tick}_main.csv")
result_df = analyze_sentiments_with_aliases(input_df, pipe, nlp, aliases[tick], truncate_or_split="split")
result_df.to_csv(f"{new_directory}/{tick}_total_sentiments.csv", index=False)
files.download(f"{new_directory}/{tick}_total_sentiments.csv")

/usr/local/lib/python3.10/dist-packages/thinc/shims/pytorch.py:114: FutureWarning: `torch.cuda.amp.autocast(args...)` is deprecated. Please use `torch.amp.autocast('cuda', args...)` instead.
  with torch.cuda.amp.autocast(self._mixed_precision):
